In [23]:
import numpy as np 
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
    roc_curve,
    roc_auc_score
)

In [48]:
# df = pd.read_csv('../../Data/heart_2022_with_nans.csv')

# sample 500 rows from df
# final = df.dropna()

# final['HadHeartDisease'] = final.apply(lambda row: 'Yes' if 'Yes' in [row['HadHeartAttack'], 
#                                                                           row['HadStroke'], 
#                                                                           row['HadAngina']] else 'No', 
#                                                                           axis=1)

# # only keep columns that contain yes or no values
# final = final[['PhysicalHealthDays', 'MentalHealthDays', 
#               'PhysicalActivities','SleepHours', 
#               'HadHeartAttack', 'HadAngina', 
#               'HadStroke', 'HadAsthma',
#               'HadSkinCancer', 'HadCOPD', 
#               'HadDepressiveDisorder', 'HadKidneyDisease',
#               'HadArthritis','HadHeartDisease']]
# final

<ipython-input-48-829ef29e1041>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['HadHeartDisease'] = final.apply(lambda row: 'Yes' if 'Yes' in [row['HadHeartAttack'],


,PhysicalHealthDays,MentalHealthDays,PhysicalActivities,SleepHours,HadHeartAttack,HadAngina,HadStroke,HadAsthma,HadSkinCancer,HadCOPD,HadDepressiveDisorder,HadKidneyDisease,HadArthritis,HadHeartDisease
342,4.0,0.0,Yes,9.0,No,No,No,No,No,No,No,No,Yes,No
343,0.0,0.0,Yes,6.0,No,No,No,No,No,No,No,No,Yes,No
345,0.0,0.0,No,8.0,No,No,No,No,No,No,No,No,Yes,No
346,5.0,0.0,Yes,9.0,No,No,No,No,Yes,No,Yes,No,Yes,No
347,3.0,15.0,Yes,5.0,No,No,No,No,No,No,No,No,Yes,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445117,0.0,0.0,Yes,6.0,No,No,No,No,No,No,No,No,No,No
445123,0.0,7.0,Yes,7.0,No,No,No,No,No,No,Yes,No,No,No
445124,0.0,15.0,Yes,7.0,No,No,Yes,No,No,No,No,No,Yes,Yes
445128,2.0,2.0,Yes,7.0,No,No,No,No,No,No,No,No,No,No


In [32]:
# prepare data for logistic regression
def logi_reg(df, target, target_names, thresh_n):
    """You know what it does 

        Input: 
            df: pandas dataframe
            target: target column name
            target_names: the names of the target classes eg."Yes" and "No"
            thresh_n: threshold used on this dataset

        Output:
            None 
    """

    # one hot encoding on categorical features 
    features = df.drop(columns = [target])   
    df = pd.get_dummies(df, columns=features.select_dtypes(include=['object']).columns)

    # data split
    df_train, df_test = train_test_split(df, 
                                        test_size = 0.25, 
                                        random_state = 69,
                                        stratify = df[target])

    X_train = df_train.drop(columns = [target])
    y_train = df_train[target]
    X_test = df_test.drop(columns = [target])
    y_test = df_test[target]

    # print the size of the training and test set
    print (f"The threshold for removing NAs is {thresh_n}")
    print (f"Size of training set : {df_train.shape[0]} rows , {df_train.shape[1]} columns")
    print (f"Size of testing set : {df_test.shape[0]} rows , {df_test.shape[1]} columns")

    # fit logistic regression model with elastic net regularization
    log_reg = LogisticRegression(penalty = 'elasticnet',
                                solver = 'saga',
                                l1_ratio = 0.5,
                                max_iter = 1000)
    
    log_reg.fit(X_train, y_train)

    # predict on test set
    y_test_pred = log_reg.predict(X_test)
    y_pred_proba = log_reg.predict_proba(X_test)[:,1]

    # Evaluate performance
    # print('\n===============================\n'+
    #   'Classification report on test data' +
    #   '\n===============================\n')
    # print(classification_report(y_test, y_test_pred, target_names=target_names))

    print('\n===============================\n'+
        'Confusion matrix on test data' +
        '\n===============================\n')
    print(confusion_matrix(y_test, y_test_pred))
    
    print("Accuracy:", accuracy_score(y_test, y_test_pred))
    print("Precision:", precision_score(y_test, y_test_pred, pos_label='Yes'))
    print("Recall:", recall_score(y_test, y_test_pred, pos_label='Yes'))
    print("F1 Score:", f1_score(y_test, y_test_pred, pos_label='Yes'))
    print("ROC AUC:", roc_auc_score(y_test, y_pred_proba))



In [21]:
# logi_reg(final, 'HadHeartDisease', ['No', 'Yes'], 0.5)

In [30]:
df_heart_drop_imp = pd.read_csv('../../Data/df_heart_drop_imp.csv')
# # only keep columns that contain yes or no values
#df_heart_drop_imp.info()
features = df_heart_drop_imp.drop(columns = ['HadHeartDisease'])
df = pd.get_dummies(df_heart_drop_imp, columns=features.select_dtypes(include=['object']).columns)
df

,PhysicalHealthDays,MentalHealthDays,SleepHours,HeightInMeters,WeightInKilograms,HadHeartDisease,State_Alabama,State_Alaska,State_Arizona,State_Arkansas,...,PneumoVaxEver_Yes,"TetanusLast10Tdap_No, did not receive any tetanus shot in the past 10 years","TetanusLast10Tdap_Yes, received Tdap","TetanusLast10Tdap_Yes, received tetanus shot but not sure what type","TetanusLast10Tdap_Yes, received tetanus shot, but not Tdap",HighRiskLastYear_No,HighRiskLastYear_Yes,CovidPos_No,CovidPos_Tested positive using home test without a health professional,CovidPos_Yes
0,0.0,0.0,8.0,1.70,80.74,No,1,0,0,0,...,0,0,0,1,0,1,0,1,0,0
1,0.0,0.0,6.0,1.60,68.04,No,1,0,0,0,...,0,1,0,0,0,1,0,1,0,0
2,2.0,3.0,5.0,1.57,63.50,No,1,0,0,0,...,0,1,0,0,0,1,0,0,0,1
3,0.0,0.0,7.0,1.65,63.50,No,1,0,0,0,...,1,1,0,0,0,1,0,1,0,0
4,2.0,0.0,9.0,1.57,53.98,No,1,0,0,0,...,1,1,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437505,0.0,3.0,6.0,1.65,69.85,No,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1
437506,2.0,2.0,7.0,1.70,83.01,No,0,0,0,0,...,0,0,0,1,0,1,0,1,0,0
437507,30.0,30.0,5.0,1.70,49.90,No,0,0,0,0,...,0,1,0,0,0,1,0,1,0,0
437508,0.0,0.0,5.0,1.83,108.86,Yes,0,0,0,0,...,1,1,0,0,0,1,0,0,0,1


In [33]:
logi_reg(df_heart_drop_imp, 'HadHeartDisease', ['No', 'Yes'], 0)

The threshold for removing NAs is 0
Size of training set : 328132 rows , 150 columns
Size of testing set : 109378 rows , 150 columns

Confusion matrix on test data

[[95413  1511]
 [10593  1861]]
Accuracy: 0.8893378924463786
Precision: 0.5518979833926453
Recall: 0.14942990203950537
F1 Score: 0.23518261089346645
ROC AUC: 0.8378982627676469


In [34]:
df_heart_drop_00_imp = pd.read_csv('../../Data/df_heart_drop_00_imp.csv')

In [35]:
logi_reg(df_heart_drop_00_imp, 'HadHeartDisease', ['No', 'Yes'], 0)

The threshold for removing NAs is 0
Size of training set : 184516 rows , 150 columns
Size of testing set : 61506 rows , 150 columns


/Users/xiaogouzuomeng/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



Confusion matrix on test data

[[53591   860]
 [ 6018  1037]]
Accuracy: 0.8881735115273307
Precision: 0.5466526093832367
Recall: 0.14698795180722893
F1 Score: 0.23168007149240397
ROC AUC: 0.8395884226028822
